In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep



In [19]:
class KakaoMapScraper:
    '''
    카카오맵 리뷰 데이터를 크롤링하는 클래스
    '''
    def __init__(self):
        # webdriver path, 카카오맵 url 설정
        self.driver = webdriver.Chrome(executable_path="./chromedriver")
        self.url = "https://map.kakao.com"
        self.driver.implicitly_wait(1)

        self.driver.get(self.url)
        self.driver.implicitly_wait(5)
    
    def _get_place_address_list(self, keywords):
        '''
        키워드 검색 결과의 장소들의 상세보기 주소를 스크래핑하여 반환
        param: keywords (type: list)
        return: place_address_list (type: dict)
        '''
        
        # 검색창 찾기
        search_area = self.driver.find_element(By.ID, "search.keyword.query") #.find_element_by_id("search.keyword.query")
        place_address_list = {}
        
        for keyword in keywords:
            # 키워드 검색
            search_area.clear()
            search_area.send_keys(keyword)
            self.driver.find_element(By.ID, "search.keyword.submit").send_keys(Keys.ENTER)
            self.driver.implicitly_wait(2)
            
            try: # 더보기 버튼이 있을시 클릭하여 진행
                self.driver.find_element(By.ID, "info.search.place.more").send_keys(Keys.ENTER)
                self.driver.implicitly_wait(2)
                
            except: # 1페이지만 존재할 때
                place_address_list[keyword]= []
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                # 현재 페이지 장소들의 상세보기 주소를 저장
                for place in places:
                    place_address_list[keyword].append(place.select_one("a")["href"])

                continue
                
            
            # 여러 페이지 존재할 경우, 1페이지부터 장소들의 상세보기 주소를 가져옴
            try:
                place_address_list[keyword]= []
                page_num = 1
                while True:
                    try:
                        # 페이지 이동
                        self.driver.find_element(By.ID, f"info.search.page.no{page_num}").send_keys(Keys.ENTER)#  find_element_by_id(f"info.search.page.no{page_num}").send_keys(Keys.ENTER)
                        sleep(1)

                        html = self.driver.page_source
                        soup = BeautifulSoup(html, "html.parser")
                        places = soup.select("ul.placelist > li.PlaceItem.clickArea > div.info_item > div.contact.clickArea")
                        
                        # 현재 페이지 장소들의 상세보기 주소를 저장
                        for place in places:
                            place_address_list[keyword].append(place.select_one("a")["href"])

                    except:
                        break
                    
                    page_num += 1
                    
                    # 5페이지가 넘어가면 다음 버튼을 클릭
                    if page_num == 6:
                        page_num = 1
                        try:
                            self.driver.find_element(By.ID,"info.search.page.next").send_keys(Keys.ENTER)
                            self.driver.implicitly_wait(2)
                        except:
                            break
            except:
                break

        return place_address_list
    
    def get_data(self, keywords):
        '''
        각 장소별 리뷰 데이터를 크롤링
        params: keywords (type: list)
        return: review_df (type: pd.DataFrame)
        '''
        
        # 키워드 장소들의 상세보기 주소를 가져옴
        place_address_list = self._get_place_address_list(keywords)
        
        # 리뷰 데이터를 저장할 데이터프레임 생성
        review_df = pd.DataFrame(columns=["name", "average_rating", "user", "user_rating", "comment", "url", "target"])
        
        # 각 장소별 리뷰 데이터 크롤링
        for place, addresses in place_address_list.items():
            for address in addresses:
                # 각 장소의 상세보기 주소로 이동
                self.driver.get(address)
                sleep(2)

                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                
                # 장소 정보 저장(장소 이름, 분류, 평균 별점)
                place_info = soup.select_one("div.inner_place")
                #name = place_info.select_one("h2.tit_location").text
                #category = place_info.select_one("div.location_evaluation > span.txt_location").text[4:]
                average_rating = float(place_info.select_one("a.link_evaluation > span.color_b").text[:-1])
                location = self.driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]').text
                print(location)


                # 주소가 "서울 마포구"가 아닐 경우 크롤링을 진행하지 않음
                if location[:6] != "서울 마포구": continue
                
                # 1페이지의 후기부터 크롤링
                # page_num = 1
                # while True:
                #     # 후기 더보기 버튼 있을시 클릭
                #     buttons = self.driver.find_elements(By.CLASS_NAME, "btn_fold")#  .find_elements_by_class_name("btn_fold")
                #     for button in buttons:
                #         if button.is_displayed() and button.is_enabled():
                #             button.send_keys(Keys.ENTER)
                #     sleep(0.5)
                    
                #     # html 파싱
                #     html = self.driver.page_source
                #     soup = BeautifulSoup(html, "html.parser")
                #     reviews = soup.select("ul.list_evaluation > li")
                    
                #     # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                #     for review in reviews:
                #         user = review.select_one("a")["data-username"]
                #         rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                #         #print(rating_per)
                #         rating_per = rating_per["style"][6:]
                #         rating_per = rating_per[:-2]
                #         user_rating = float(rating_per)/20
                #         comment = review.select_one("div.comment_info > p.txt_comment > span").text
                        
                #         review_data = {"name": place, 
                #                        "average_rating": average_rating,
                #                        "user": user,
                #                        "user_rating": user_rating,
                #                        "comment": comment,
                #                        "url": address}
                #         review_df = review_df.append(review_data, ignore_index=True)
                    
                #     # 다음 후기 페이지가 있으면 이동
                #     try:
                #         page_num += 1
                #         self.driver.find_element_by_css_selector(f"a[data-page='{page_num}']").click()
                #         sleep(1)
                #     except: 
                #         break

                # 후기 크롤링
                while True:
                    # 후기 더보기 버튼 끝까지 클릭
                    button = self.driver.find_element(By.CLASS_NAME, 'link_more')
                    sleep(0.5)
                    btn_text = button.text
                    
                    if btn_text == "메뉴 더보기":
                        button = self.driver.find_elements(By.CLASS_NAME, 'link_more')
                        sleep(0.5)
                        btn_text = button[1].text
                        button = button[1]

                    if btn_text == "후기 접기":
                        break

                    button.click()
                
                # html 파싱
                html = self.driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                reviews = soup.select("ul.list_evaluation > li")
                
                # 각 후기별 정보 저장(작성자 이름, 작성자 별점, 작성자 후기)
                for review in reviews:
                    user = review.select_one("a")["data-username"]
                    rating_per = review.select_one("div.star_info > div > span > span")# > div.grade_star size_s > span.ico_star star_rate > span.ico_star inner_star")
                    #print(rating_per)
                    rating_per = rating_per["style"][6:]
                    rating_per = rating_per[:-2]
                    user_rating = float(rating_per)/20
                    comment = review.select_one("div.comment_info > p.txt_comment > span").text
                    # if user_rating > 3:
                    #     target = 'good'
                    # elif user_rating == 3:
                    #     target = 'soso'
                    # else :
                    #     target = 'bad'
                    target = 1 if user_rating >3 else 0
                    review_data = {"name": place, 
                                    "average_rating": average_rating,
                                    "user": user,
                                    "user_rating": user_rating,
                                    "comment": comment,
                                    "url": address,
                                    "target": target}
                    review_df = review_df.append(review_data, ignore_index=True)

        return review_df

In [20]:
scraper = KakaoMapScraper()

/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path="./chromedriver")


In [21]:
keywords = ["후계동", "카미야", "카레시", "국시와 가래떡", "지로우라멘마포", "하카타분코홍대", "가미우동마포", "요이동", "스시히카리마포양화로","노티드연남","맛이차이나홍대", "윤씨밀방", "칸다소바홍대", "발바리네홍대", "바다회사랑 2호점", "스케줄합정", "디어리스트연남", "정돈마포","비스트로사랑방", "우와홍대", "공감합정점", "흠식당홍대", "산더미불고기서교", "마라공방홍대", "아오이토리홍대", "연남토마홍대", "버터밀크홍대", "감성타코홍대", "프리모바치오바치홍대", "더다이닝랩홍대", "스시소라마포점", "옥동식서교", "혼가츠홍대"]

df = scraper.get_data(keywords)

서울 마포구 독막로 76 (우)04074


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 와우산로21길 28-6 지하1층 (우)04040


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 독막로9길 31 (우)04048


...

/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 양화로7길 44-10 1층 (우)04035


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed fro

서울 마포구 와우산로21길 36-6 102호 (우)04040


/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  review_df = review_df.append(review_data, ignore_index=True)
/var/folders/m3/9rlp04l902592zqcywkn5ppm0000gn/T/ipykernel_50471/4163420982.py:207: FutureWarning: The frame.append method is deprecated and will be removed fro

In [22]:
df

,name,average_rating,user,user_rating,comment,url,target
0,후계동,4.5,풀내음,5.0,,https://place.map.kakao.com/1183957472,1
1,후계동,4.5,ㅇㅇ,5.0,❤️❤️❤️❤️❤️,https://place.map.kakao.com/1183957472,1
2,후계동,4.5,핑구,5.0,오래 해주세요🙏,https://place.map.kakao.com/1183957472,1
3,후계동,4.5,:ᴅ,3.0,웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...,https://place.map.kakao.com/1183957472,0
4,후계동,4.5,Vin,5.0,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!,https://place.map.kakao.com/1183957472,1
...,...,...,...,...,...,...,...
3125,혼가츠홍대,2.9,hj,2.0,,https://place.map.kakao.com/22895023,0
3126,혼가츠홍대,2.9,말,4.0,,https://place.map.kakao.com/22895023,1
3127,혼가츠홍대,2.9,이정민,4.0,,https://place.map.kakao.com/22895023,1
3128,혼가츠홍대,2.9,sp,2.0,,https://place.map.kakao.com/22895023,0


In [23]:
# #target 값 설정
# y = df.shape[0]*[0]
# target = pd.Series(y, name='target')

# df = pd.concat([df, target], axis=1)
# df

df.info(verbose=True)
df["comment"].isnull().sum()
df["comment"][722]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3130 entries, 0 to 3129
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            3130 non-null   object 
 1   average_rating  3130 non-null   float64
 2   user            3130 non-null   object 
 3   user_rating     3130 non-null   float64
 4   comment         3130 non-null   object 
 5   url             3130 non-null   object 
 6   target          3130 non-null   object 
dtypes: float64(2), object(5)
memory usage: 171.3+ KB


'튀김은 내 스타일 아니지만, 우동은 진짜 맛나요!!!웨이팅만 아니면, 일주일에 한번은 가고싶은 맛~'

In [24]:
df.to_csv("홍대_맛집_리뷰_데이터_ver5.csv")
#df.to_excel("홍대_맛집_리뷰_데이터.xlsx")